# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [55]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_pull = 'output_data/weather_data_df.csv'
wmap_df = pd.read_csv(weather_data_pull)

wmap_df = wmap_df.drop(columns='Unnamed: 0')

wmap_df.head()
wmap_df.dropna()

,city,lat,lng,max_temp,humidity,cloudiness,windspeed,country,date,max_temp_cel
1,staroutkinsk,57.2302,59.3348,288.34,96.0,100.0,2.57,RU,1.624125e+09,15.19
2,busselton,-33.6500,115.3333,286.51,85.0,100.0,4.13,AU,1.624124e+09,13.36
3,carnarvon,-24.8667,113.6333,283.19,44.0,40.0,1.54,AU,1.624124e+09,10.04
4,mataura,-46.1927,168.8643,273.97,98.0,23.0,1.31,NZ,1.624124e+09,0.82
5,hermanus,-34.4187,19.2345,294.21,49.0,30.0,3.26,ZA,1.624124e+09,21.06
...,...,...,...,...,...,...,...,...,...,...
606,eydhafushi,5.1033,73.0708,301.46,75.0,59.0,6.39,MV,1.624124e+09,28.31
607,soligalich,59.0778,42.2864,293.41,72.0,3.0,2.64,RU,1.624125e+09,20.26
608,iquitos,-3.7481,-73.2472,302.17,74.0,75.0,3.60,PE,1.624125e+09,29.02
609,san andres,12.5847,-81.7006,304.16,74.0,75.0,6.17,CO,1.624125e+09,31.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
gmaps.configure(api_key=g_key)

location = wmap_df[['lat','lng']].astype(float)
weight = wmap_df['humidity'].astype(float)

location = location.dropna()
weight = weight.dropna()


In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
hotel_df = wmap_df[(wmap_df['max_temp_cel']>19) & (wmap_df['windspeed']<20) & (wmap_df['cloudiness']<20)]
hotel_df.head()

,city,lat,lng,max_temp,humidity,cloudiness,windspeed,country,date,max_temp_cel
16,kamra,33.7473,73.5111,299.79,33.0,13.0,3.97,PK,1.624125e+09,26.64
19,waingapu,-9.6567,120.2641,297.14,90.0,15.0,2.90,ID,1.624125e+09,23.99
35,saint george,37.1041,-113.5841,310.12,17.0,1.0,0.89,US,1.624124e+09,36.97
39,marand,38.4329,45.7749,301.34,31.0,18.0,4.13,IR,1.624125e+09,28.19
51,taltal,-25.4000,-70.4833,292.71,54.0,0.0,2.10,CL,1.624125e+09,19.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
hotel_df


,city,lat,lng,max_temp,humidity,cloudiness,windspeed,country,date,max_temp_cel,Hotel Name
16,kamra,33.7473,73.5111,299.79,33.0,13.0,3.97,PK,1.624125e+09,26.64,Cheraas
19,waingapu,-9.6567,120.2641,297.14,90.0,15.0,2.90,ID,1.624125e+09,23.99,Waingapu
35,saint george,37.1041,-113.5841,310.12,17.0,1.0,0.89,US,1.624124e+09,36.97,St. George
39,marand,38.4329,45.7749,301.34,31.0,18.0,4.13,IR,1.624125e+09,28.19,Marand
51,taltal,-25.4000,-70.4833,292.71,54.0,0.0,2.10,CL,1.624125e+09,19.56,Taltal
...,...,...,...,...,...,...,...,...,...,...,...
588,umm lajj,25.0213,37.2685,303.65,43.0,0.0,3.64,SA,1.624124e+09,30.50,Umluj
589,saint-joseph,-21.3667,55.6167,292.71,73.0,14.0,4.54,RE,1.624125e+09,19.56,Saint Joseph
590,myshkin,57.7897,38.4545,294.64,81.0,11.0,3.12,RU,1.624125e+09,21.49,Myshkin
593,redlands,34.0556,-117.1825,316.73,43.0,1.0,0.00,US,1.624125e+09,43.58,Redlands


In [79]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params ={
    'radius':5000,
    'key': g_key,
    'type': 'hotel'
}

for index, row in hotel_df.iterrows():

    lat_ = row['lat']
    lng_ = row['lng']
    
    params['location'] = f'{lat_},{lng_}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params=params)
    
    hotel_map_df = response.json()
    
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_map_df['results'][0]['name']

    except (KeyError, IndexError, RuntimeError, TypeError, NameError):
        print('No hotel found. skipping...')


C:\Users\User1\.conda\envs\SRSpython\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [81]:
hotel_df

,city,lat,lng,max_temp,humidity,cloudiness,windspeed,country,date,max_temp_cel,Hotel Name
16,kamra,33.7473,73.5111,299.79,33.0,13.0,3.97,PK,1.624125e+09,26.64,Cheraas
19,waingapu,-9.6567,120.2641,297.14,90.0,15.0,2.90,ID,1.624125e+09,23.99,Waingapu
35,saint george,37.1041,-113.5841,310.12,17.0,1.0,0.89,US,1.624124e+09,36.97,St. George
39,marand,38.4329,45.7749,301.34,31.0,18.0,4.13,IR,1.624125e+09,28.19,Marand
51,taltal,-25.4000,-70.4833,292.71,54.0,0.0,2.10,CL,1.624125e+09,19.56,Taltal
...,...,...,...,...,...,...,...,...,...,...,...
588,umm lajj,25.0213,37.2685,303.65,43.0,0.0,3.64,SA,1.624124e+09,30.50,Umluj
589,saint-joseph,-21.3667,55.6167,292.71,73.0,14.0,4.54,RE,1.624125e+09,19.56,Saint Joseph
590,myshkin,57.7897,38.4545,294.64,81.0,11.0,3.12,RU,1.624125e+09,21.49,Myshkin
593,redlands,34.0556,-117.1825,316.73,43.0,1.0,0.00,US,1.624125e+09,43.58,Redlands


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [91]:
hotels = hotel_df['Hotel Name'].tolist()

In [96]:
# ##Add marker layer ontop of heat map

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0,100,0,0)', 
    stroke_color='rgba(0,0,100,0)', scale=3, 
    info_box_content=[f'{hotel_info}' for hotel in hotels]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig



Figure(layout=FigureLayout(height='420px'))

In [95]:
### Display figure

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))